# The Battle of Neighborhoods

## Introduction 
This is the notebook that belongs to the official report. 

#### Business problem - Recap
As mentioned in the report, John will go to Columbia University and he needs a place to stay. 
He asked my help to find the best neighborhood, given the folowing requirements. 

John and his girlfriends prefers to live in a neighborhood with a: 
- Bakery
- Bar 
- Breakfast Spot
- Café
- Cocktail Bar
- Coffee Shop 
- Gym / Fitness Center 
- Sandwich Place
- Shopping Mall
- Sushi Restaurant
- Pizza Place
- Yoga Studio

## Step 1
__In the first step we are going to import all libraries that we need. Then we are going to load the data of New York. After that we put all the data in a dataframe. The last step is to extract all unneeded data so that we only have the Manhattan data.__

#### Import libraries 


In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

from bs4 import BeautifulSoup # import recommended html parsing library

print('Libraries imported.')

Libraries imported.


__Now that we have imported all the libraries, we are going to download and load the New York Dataset that is already placed on the server.__ 

In [8]:
#Download data
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

#Load data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


__For our business problem, we want to retrieve a list of the neighborhoods of Manhattan. 
So first we examen the results of the json.__ 

In [9]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

__You can see that all relevant information is in the features key, since that is a list of the neighborhoods. We make a new variable that includes this data and we create an empty dataframe.__

In [11]:
#Create variable for features 
neighborhoods_data = newyork_data['features']

# Create empty dataframe 
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


__Now we are going to fill this dataframe with the retrieved information from the json.__ 

In [180]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

#Let's have a look at the top results of the dataframe
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


__Since we only want the data of Manhattan, we slice the dataframe and create a new dataframe that only contains the data of Manhattan.__

In [181]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


## Step 2

__In this step we are going to make a map of Manhattan.__

__First we use Geopy library to get the latitude and longitude values of Manhattan.__

In [182]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


__Next we make a map of Manhattan with the neighborhoods superimposed on top by using Folium.__ 

In [183]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], 
    manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

## Step 3
__In this step we are going to start utilizing the Foursquare API. We retreive all data that we need for this assignment, we clean the data and we store it into a dataframe for further analysis.__

In [184]:
#Define Foursquare Credentials and Version 
CLIENT_ID = 'RNOZQCLVFPDRIZTKLDYFOPDRHJDZD0WAXRIYSECCTPOP2VMF' # your Foursquare ID
CLIENT_SECRET = 'XB3BVV05SCJF3LYM5YWXZFJAKUOSKSEYT44IF31H11SWCVQN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: RNOZQCLVFPDRIZTKLDYFOPDRHJDZD0WAXRIYSECCTPOP2VMF
CLIENT_SECRET:XB3BVV05SCJF3LYM5YWXZFJAKUOSKSEYT44IF31H11SWCVQN


__Let's first explore the neighborhoods in our dataframe. We begin with the first neighborhood's name.__ 

In [95]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

__Let's get the neighborhood's latitude and longitude values.__

In [94]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


__Let's use Foursquare to retrieve the top 100 venues in a radius of 500 meter.__

In [98]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
# display URL
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=RNOZQCLVFPDRIZTKLDYFOPDRHJDZD0WAXRIYSECCTPOP2VMF&client_secret=XB3BVV05SCJF3LYM5YWXZFJAKUOSKSEYT44IF31H11SWCVQN&v=20180605&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

__Next we send the GET request, clean the json and structure it into a pandas dataframe.__

In [99]:
# Send the GET request
results = requests.get(url).json()

In [100]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [101]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Land & Sea Restaurant,Seafood Restaurant,40.877885,-73.905873


__Let's check how many venues were returned by Foursquare for this neighborhood.__

In [102]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


In [166]:
nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Land & Sea Restaurant,Seafood Restaurant,40.877885,-73.905873


__Now we create the getNearbyVenues function that we will use to retrieve the information from all neighborhoods of Manhattan.__

In [107]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

__We use the following code to run the above function for each neighborhood and create a new dataframe.__

In [108]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


## Step 4
__In this step we are going to analyze all data we have so far to see what we already have and how this data can help us with the assignment.__

__Let's first count all the unique neighborhoods in Manhattan.__

In [120]:
neigh_list = list(manhattan_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside Manhattan:')
print(len(neigh_list))

Number of Neighborhoods inside Manhattan:
40


__Let's check the size and head of the dataframe.__ 

In [109]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3313, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant


__Let's find out how many unique venues there are in our dataset.__

In [122]:
print('In this dataset there are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

In this dataset there are 330 uniques categories.


__And let's make a list of all the different categories.__

In [123]:
list(manhattan_venues['Venue Category'].unique())

['Pizza Place',
 'Yoga Studio',
 'Diner',
 'Coffee Shop',
 'Seafood Restaurant',
 'Donut Shop',
 'Tennis Stadium',
 'Gym',
 'Department Store',
 'Video Game Store',
 'Discount Store',
 'Pharmacy',
 'Supplement Shop',
 'Steakhouse',
 'Ice Cream Shop',
 'Shoe Store',
 'American Restaurant',
 'Bank',
 'Sandwich Place',
 'Kids Store',
 'Shopping Mall',
 'Deli / Bodega',
 'Chinese Restaurant',
 'General Entertainment',
 'Cocktail Bar',
 'Greek Restaurant',
 'English Restaurant',
 'Museum',
 'Hotel',
 'Noodle House',
 'Garden Center',
 'Spa',
 'Bubble Tea Shop',
 'Bar',
 'Tea Room',
 'Roof Deck',
 'New American Restaurant',
 'Indie Movie Theater',
 'Salon / Barbershop',
 'Malay Restaurant',
 'Furniture / Home Store',
 'Hotpot Restaurant',
 'Bike Shop',
 'Vietnamese Restaurant',
 'Dim Sum Restaurant',
 'Spanish Restaurant',
 'Italian Restaurant',
 'Historic Site',
 'Asian Restaurant',
 'Jewelry Store',
 'Grocery Store',
 'Organic Grocery',
 'Record Shop',
 'Music Venue',
 'Nail Salon',
 'Cosm

__Now we use One Hot Encoding to see which venues are present in the neighborhoods.__

In [136]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social 

__Next we make a list of the required venues. John prefers a neighborhood with a:__
- Bakery
- Bar 
- Breakfast Spot
- Café
- Cocktail Bar
- Coffee Shop 
- Gym / Fitness Center 
- Sandwich Place
- Shopping Mall
- Sushi Restaurant
- Pizza Place
- Yoga Studio

In [179]:
# Create a list of the required venues
required_venues = [
    'Neighborhood', 
    'Bakery', 
    'Bar', 
    'Breakfast Spot', 
    'Café', 
    'Cocktail Bar', 
    'Coffee Shop', 
    'Gym / Fitness Center', 
    'Sandwich Place',
    'Shopping Mall',
    'Sushi Restaurant',
    'Pizza Place', 
    'Yoga Studio']
required_venues

['Neighborhood',
 'Bakery',
 'Bar',
 'Breakfast Spot',
 'Café',
 'Cocktail Bar',
 'Coffee Shop',
 'Gym / Fitness Center',
 'Sandwich Place',
 'Shopping Mall',
 'Sushi Restaurant',
 'Pizza Place',
 'Yoga Studio']

__Let's update our dataframe so that it only shows the required venues and group the data by neighborhoods.__

In [167]:
manhattan_onehot = manhattan_onehot[required_venues].groupby('Neighborhood').sum()
manhattan_onehot.head()

,Bakery,Bar,Breakfast Spot,Café,Cocktail Bar,Coffee Shop,Gym / Fitness Center,Sandwich Place,Shopping Mall,Sushi Restaurant,Pizza Place,Yoga Studio
Neighborhood,,,,,,,,,,,,
Battery Park City,1,0,0,0,0,8,1,2,2,1,2,0
Carnegie Hill,2,3,1,4,1,5,2,0,0,1,6,3
Central Harlem,0,1,0,1,0,0,2,0,0,0,1,0
Chelsea,4,1,1,1,1,7,1,1,0,1,1,0
Chinatown,3,2,0,0,4,2,0,2,0,0,1,1


__We use K-Means to cluster the neighborhoods into 5 clusters.__

In [171]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(manhattan_onehot)

# showing centers of each cluster
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = manhattan_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Bakery,Bar,Breakfast Spot,Café,Cocktail Bar,Coffee Shop,Gym / Fitness Center,Sandwich Place,Shopping Mall,Sushi Restaurant,Pizza Place,Yoga Studio,Total Sum
G1,1.555556,3.000000,0.333333,1.666667,2.000000,4.222222,1.444444,1.000000,0.000000e+00,2.444444,2.777778,6.666667e-01,21.111111
G4,3.250000,1.000000,0.000000,1.500000,1.000000,2.000000,5.500000,1.000000,0.000000e+00,1.250000,0.500000,2.500000e+00,19.500000
G2,1.166667,1.166667,0.166667,1.333333,0.666667,6.666667,1.500000,1.666667,3.333333e-01,0.333333,1.333333,1.110223e-16,16.333333
G3,2.625000,0.687500,0.187500,2.562500,1.562500,2.437500,0.625000,1.187500,1.387779e-17,1.000000,1.937500,1.250000e+00,16.062500
G5,0.000000,1.200000,0.000000,0.400000,0.200000,1.000000,0.400000,0.600000,2.000000e-01,0.200000,0.600000,2.000000e-01,5.000000


## Step 6
__In this final step we analyse the results to answer John's question.__

__The best cluster is Group 5. Now we need to find the corresponding neighborhoods in each group. Therefor we combine the kmeans labels with the original dataframe.__

In [174]:
manhattan_data = pd.DataFrame([manhattan_onehot.index, 1 + kmeans.labels_]).T
manhattan_data.columns = ['Neighborhood', 'Group']
manhattan_data

,Neighborhood,Group
0,Battery Park City,2
1,Carnegie Hill,1
2,Central Harlem,5
3,Chelsea,2
4,Chinatown,3
5,Civic Center,4
6,Clinton,2
7,East Harlem,3
8,East Village,1
9,Financial District,2


__So let's recap:__
- Group 5 had the best results for John 
- John wants to know which neighborhoods suits his preferences best 

__So let's check the neighborhoods of group 5.__

In [175]:
manhattan_data[manhattan_data['Group'] == 5]

,Neighborhood,Group
2,Central Harlem,5
21,Manhattanville,5
22,Marble Hill,5
28,Roosevelt Island,5
30,Stuyvesant Town,5


## Final conclusion

__Since Manhattanville is closeby the Columbia University, we will suggest John to start his search for a house in that neighborhood!__